In [ ]:
# Library Imports
import pandas as pd
import math

In [154]:
# decision Matrix (D)

data = {
    'X1': [2, 2.5, 1.8, 2.2],
    'X2': [1500, 2700, 2000, 1800],
    'X3': [20000, 18000, 21000, 20000],
    'X4': [5.5, 6.5, 4.5, 5],
    'X5': [5, 3, 7, 5],
    'X6': [9, 5, 7, 5],
}

# Creating DataFrame with alternatives A1 to A4
D = pd.DataFrame(data, index=['A1', 'A2', 'A3', 'A4'])

# Criteria Types
criteria_types = pd.DataFrame([["+", "+", "+", "-", "+", "+"]], columns=D.columns)

# Weights
weights = pd.DataFrame([[0.2, 0.1, 0.1, 0.1, 0.2, 0.3]], columns=D.columns)

In [155]:
#Normalized Matrix (N)
N = D.copy().astype(float)
for col_name in N.columns:
  col_sum = (N[col_name] ** 2).sum()
  N[col_name] /= math.sqrt(col_sum)


N.round(2)

,X1,X2,X3,X4,X5,X6
A1,0.47,0.37,0.51,0.51,0.48,0.67
A2,0.58,0.66,0.46,0.60,0.29,0.37
A3,0.42,0.49,0.53,0.41,0.67,0.52
A4,0.51,0.44,0.51,0.46,0.48,0.37


In [156]:
# Weighted Matrix (V)

V = N * weights.values
V.round(3)

,X1,X2,X3,X4,X5,X6
A1,0.093,0.037,0.051,0.051,0.096,0.201
A2,0.117,0.066,0.046,0.060,0.058,0.112
A3,0.084,0.049,0.053,0.041,0.135,0.157
A4,0.103,0.044,0.051,0.046,0.096,0.112


In [157]:
# Sets


# Concordance Set
c_ij = {i: {} for i in V.index}
# Discordance Set
d_ij = {i: {} for i in V.index}

for i in V.index:
  for j in V.index:
    if i ==j :
      continue

    d_ij[i][j] = []
    c_ij[i][j] = []

    for attr in V.columns:

      # Beneficial
      if criteria_types.loc[0, attr] == "+":
        if V.loc[i, attr] >= V.loc[j, attr]:
          c_ij[i][j].append(attr)
        else:
          d_ij[i][j].append(attr)
      # Detrimental
      elif criteria_types.loc[0, attr] == "-":
        if V.loc[i, attr] <= V.loc[j, attr]:
          c_ij[i][j].append(attr)
        else:
          d_ij[i][j].append(attr)


# c_ij and d_ij print
for i in V.index:
    for j in V.index:
        if i == j:
            continue
        if d_ij[i][j]:
            print(f"D({i}, {j}): {', '.join(d_ij[i][j])}")
        if c_ij[i][j]:
            print(f"C({i}, {j}): {', '.join(c_ij[i][j])}")
        print("*****")

D(A1, A2): X1, X2
C(A1, A2): X3, X4, X5, X6
*****
D(A1, A3): X2, X3, X4, X5
C(A1, A3): X1, X6
*****
D(A1, A4): X1, X2, X4
C(A1, A4): X3, X5, X6
*****
D(A2, A1): X3, X4, X5, X6
C(A2, A1): X1, X2
*****
D(A2, A3): X3, X4, X5, X6
C(A2, A3): X1, X2
*****
D(A2, A4): X3, X4, X5
C(A2, A4): X1, X2, X6
*****
D(A3, A1): X1, X6
C(A3, A1): X2, X3, X4, X5
*****
D(A3, A2): X1, X2
C(A3, A2): X3, X4, X5, X6
*****
D(A3, A4): X1
C(A3, A4): X2, X3, X4, X5, X6
*****
D(A4, A1): X6
C(A4, A1): X1, X2, X3, X4, X5
*****
D(A4, A2): X1, X2
C(A4, A2): X3, X4, X5, X6
*****
D(A4, A3): X2, X3, X4, X5, X6
C(A4, A3): X1
*****


In [173]:
# Concordance Matrix (C)
C = pd.DataFrame(0.00, index=V.index, columns=V.index)

# Discordance Matrix
D = pd.DataFrame(0.00, index=V.index, columns=V.index)
for i in C.index:
  for j in C.index:
    if i ==j :
      continue
    for attr in c_ij[i][j]:
      C.loc[i, j] += weights.loc[0, attr]
    temp = []
    for attr in d_ij[i][j]:
      temp.append(abs(V.loc[i, attr] - V.loc[j, attr]))
    D.loc[i, j] = max(temp) / (V.loc[i] - V.loc[j]).abs().max()

,A1,A2,A3,A4
A1,0.0,0.7,0.5,0.6
A2,0.3,0.0,0.3,0.6
A3,0.5,0.7,0.0,0.8
A4,0.7,0.7,0.2,0.0


In [185]:
# Thresholds
c_bar = 0
d_bar = 0
for i in C.index:
  for j in C.index:
    if i ==j:
      continue
    c_bar += C.loc[i, j]
    d_bar += D.loc[i, j]
m = len(C.index)
c_bar /= m * (m - 1)
d_bar /= m * (m - 1)

In [186]:
# F
# Concordance Dominance Matrix
F = C.copy()
for i in F.index:
  for j in F.index:
    if i ==j:
      continue
    if C.loc[i, j] >= c_bar:
      F.loc[i, j] = 1
    else:
      F.loc[i, j] = 0

In [187]:
# G
# Discordance Dominance Matrix
G = D.copy()
for i in G.index:
  for j in G.index:
    if i ==j:
      continue
    if D.loc[i, j] <= d_bar:
      G.loc[i, j] = 1
    else:
      G.loc[i, j] = 0
G

,A1,A2,A3,A4
A1,0.0,1.0,0.0,1.0
A2,0.0,0.0,0.0,0.0
A3,0.0,1.0,0.0,1.0
A4,0.0,1.0,0.0,0.0


In [192]:
# Aggregate Matrix
# E
E = C.copy()
for i in E.index:
  for j in E.index:
    if i ==j:
      continue
    E.loc[i, j] = G.loc[i, j] * F.loc[i, j]
E

,A1,A2,A3,A4
A1,0.0,1.0,0.0,1.0
A2,0.0,0.0,0.0,0.0
A3,0.0,1.0,0.0,1.0
A4,0.0,1.0,0.0,0.0


In [197]:
# Ranking the alternatives
rank = {}
for i in E.index:
  rank[i] = E.loc[i].sum()

rank = dict(sorted(rank.items(), key=lambda item: item[1], reverse=True))

max_rank = max(rank.values())
print("Suggested Set:")
for i in rank.keys():
  if rank[i] == max_rank:
    print(i)

Suggested Set:
A1
A3
